In [ ]:
import json
from clients.mcp_client import MCPClient
from clients.llm_client import ModelRaw

In [26]:
mcp_client = MCPClient('http://127.0.0.1:7860/gradio_api/mcp/sse')
tools_ = await mcp_client.list_tools()
tools_.tools

[Tool(name='duckduckgo_search', description='Search query in internet via DuckDuckGo Search enginre', inputSchema={'type': 'object', 'properties': {'search_query': {'type': 'string', 'description': 'The input text to search through'}}}, annotations=None),
 Tool(name='reddit_search', description='Search query in reddit, return fetched posts with comment and information', inputSchema={'type': 'object', 'properties': {'search_query': {'type': 'string', 'description': 'The input text to search through'}}}, annotations=None),
 Tool(name='parse_url', description='Parse indeep info from web page. Get url and return all information than page have', inputSchema={'type': 'object', 'properties': {'url': {'type': 'string', 'description': 'url of web resource'}}}, annotations=None)]

In [3]:
# format tools for model
json_tools = [{'type':'function','function':json.loads(tool.model_dump_json())} for tool in tools_.tools]

In [4]:
json_tools[1]

{'type': 'function',
 'function': {'name': 'reddit_search',
  'description': 'Search query in reddit, return fetched posts with comment and information',
  'inputSchema': {'type': 'object',
   'properties': {'search_query': {'type': 'string',
     'description': 'The input text to search through'}}},
  'annotations': None}}

In [ ]:
model = ModelRaw('http://0.0.0.0:8001/v1',
                mcp_client = mcp_client,
                SYSTEM_PROMPT='''Ты умный помошник для работы с данными. Твоя задача ответить на вопрос пользователя используя максимум доступной тебе информации.
                                Если тебе ее не хватает - обязательно используй доступные тебе инструменты.
                                Не выдумывай факты. Если есть возможность получить информацию из нескольких источников сразу - получай из всех доступных.
                                Ты можешь одновременно использовать неограниченное количество инструментов за раз.
                                Не завершай задачу пока не будешь уверен в результате.
                                Всегда составляй план перед запросом и действуй по шагам
                                ''')
await model.__init_mcp__()

### Raw answer

In [15]:
raw_answer = await model.ainvoke([{'role':'user','content':'Какие новости сейчас есть про модель Gemma 3n?'}],tools=[])

In [16]:
print(raw_answer['choices'][0]['message']['reasoning_content'])


Хорошо, пользователь спрашивает, какие новости сейчас есть про модель Gemma 3n. Сначала мне нужно понять, что такое Gemma 3n. Возможно, это версия модели Gemma, но в предыдущих данных не было упоминания о такой версии. Возможно, это опечатка или неправильное название. Нужно проверить, есть ли такая модель.

Сначала проверю, существует ли модель Gemma 3. В предыдущих данных говорилось о модели Gemma 2, которая является улучшенной версией Gemma 1. Возможно, пользователь имеет в виду Gemma 2, но написал 3n. Или, возможно, это новая версия, которая не была упомянута в предыдущих данных.

Теперь нужно проверить, есть ли информация о Gemma 3n в доступных источниках. Поскольку у меня есть доступ к инструментам, я могу использовать поисковые функции, чтобы найти последние новости. Но, возможно, в доступных данных нет информации о Gemma 3n, поэтому нужно уточнить у пользователя, возможно, это опечатка.

Также стоит проверить, есть ли связь между Gemma 3n и другими моделями, например, Llama 3, 

In [17]:
print(raw_answer['choices'][0]['message']['content'])



На текущий момент нет информации о модели "Gemma 3n" в доступных данных. Возможно, вы имеете в виду:

1. **Gemma 2** — улучшенная версия модели Gemma 1, которая была анонсирована в 2023 году. Она предлагает улучшенные параметры, производительность и поддержку множества языков.

2. **Llama 3** — последняя версия серии Llama, разработанная Meta. Она имеет более крупные параметры и улучшенные функции, но не связана напрямую с Gemma.

Рекомендую уточнить, возможно, вы имеете в виду другую модель или версию. Если нужно, могу проверить актуальные новости через внешние источники.


# MCP Powered Model

### simple call

In [18]:
import asyncio

In [23]:
# Function to parallel call functionss
async def parallel_func_calls(requested_tools:list):
    func_answers = await asyncio.gather(*[mcp_client.invoke_tool(
                                                    name = tool['function']['name'],
                                                    arguments = json.loads(tool['function']['arguments']),
                                                    )
                                        for tool in requested_tools])
    print([tool['function']['name'] for tool in requested_tools])
    return {'role':'tool','content':'\n\n'.join([func.content for func in func_answers])}


In [36]:
func_answers = await parallel_func_calls(
    requested_tools=[
        {"function": {
                "name": "reddit_search",
                "arguments":json.dumps({"search_query": "Who is Post Malown?"})}},
        {"function": {
                "name": "duckduckgo_search",
                "arguments":json.dumps({"search_query": "Who is Post Malown?"}),
            }
        },
    ]
)

['reddit_search', 'duckduckgo_search']


In [39]:
print(func_answers['content'])

{"type":"text","text":"{'headers': ['index', 'comments', 'score', 'comment_count'], 'data': [['PvE Retribution Paladin Guide: How to get your guild to take you seriously.', [\"tl;dr\\n\\nYou can raid as a Ret paladin son, but boy you're gonna have to suck a lot of dicks.\", 'make friends, contribute, be nice', 'No self respecting man should ever kiss this much ass.', \"Oh dear, where to begin.\\n\\nYou focus entirely on Nightfall. Which sucks. Nearly nobody wants to raid as a ret paladin and just be the Nightfall bot, who works his ass off and doesn't get any gear unless nobody needs it anymore because their only purpose is applying Nightfall. If that's good enough for you, fine, but making a guide, saying that this is the ONLY way you can play ret in pve and every ret must go for this, is ludicrous. Nightfall is necessary to have and if you feel the need you should switch to it, as it provides a raid dps increase when raiding as AP ret, yes, but using it all the time is a no go for me

### Main soup from different loop

In [66]:
request = 'Расскажи мне краткую историю создания Властелина колец. Используй как можно больше источников.'

messages = [
    {'role':'user','content':request}
]

n = 0
tools = None
finish_reason = 'initial_question'
while finish_reason not in ('stop','length'):

    response = await model.ainvoke(messages,tools=tools)
    finish_reason = response['choices'][0]['finish_reason']
    print(f'Step {n} : {finish_reason}')
        

    if response['choices'][0]['message']['tool_calls']: 
        messages.append(await parallel_func_calls(response['choices'][0]['message']['tool_calls']))  
    else:
        messages.append({'role':'assistant','content':response['choices'][0]['message']['content']})
                    
    n+=1

Step 0 : tool_calls
['duckduckgo_search', 'reddit_search']
Step 1 : stop


In [67]:
print(response['choices'][0]['message']['reasoning_content'])


Хорошо, мне нужно рассказать краткую историю создания "Властелина колец" используя как можно больше источников. Давайте посмотрю на информацию, которую я получил.

Сначала, из первого ответа видно, что история создания начинается в 1937 году и заканчивается в 1954-1955 годах. Джон Р. Р. Толкин начал разрабатывать мир Средиземья задолго до написания романа. В 1930-х годах он начал создавать свой фантастический мир, что стало основой для "Властелина колец". Также упоминается, что роман был написан в 1954 году, но процесс создания занял несколько десятилетий.

Вот еще детали: Толкин был лингвистом и преподавателем в Оксфордском университете. Он создал множество элементов, включая языки, исторические сюжеты, которые позже были использованы в романе. Также есть информация о том, что он разрабатывал идею в 1930-х, а написание романа заняло годы, с перерывами.

Нужно объединить эти сведения, проверить, нет ли противоречий, и структурировать историю по времени. Также можно добавить, что роман

In [68]:
print(response['choices'][0]['message']['content'])



Краткая история создания "Властелина колец" основана на данных из нескольких источников:

1. **Начало разработки мира (1930-е годы):**  
   Джон Р. Р. Толкин начал создавать фантастический мир Средиземья в 1930-х годах. Он был лингвистом и преподавателем в Оксфордском университете, что повлияло на его интерес к языкам и истории. В этот период он разработал основы саги, включая языки (например, Келдорийский), исторические сюжеты и кultureльные элементы, которые позже стали частью романа.

2. **Написание романа (1954–1955 годы):**  
   Основное написание "Властелина колец" началось в 1954 году, но процесс создания занимал несколько десятилетий. Толкин не сразу написал роман — он постепенно объединил разработанные элементы, включая сагу о Средиземье, языки, исторические события и персонажей. Роман был опубликован в 1954–1955 годах издательством Allen & Unwin.

3. **Долгий и сложный процесс:**  
   История создания романа описана как многолетний процесс. Толкин не был "однажды" автором —

### Embedded Model

In [ ]:
answer = await model.precess_request('Какие новости сейчас есть про модель Gemma 3n?',steps_limit=4)

Step 0 : tool_calls
['duckduckgo_search']
Step 1 : stop


In [71]:
print(answer['choices'][0]['message']['reasoning_content'])


Хорошо, мне нужно обработать информацию о новостях про модель Gemma 3n, которые были получены через search. Сначала я должен проанализировать данные, которые были возвращены из search. 

Сначала посмотрю на структуру данных. В ответе есть таблица с несколькими новостями, каждая из которых имеет заголовок, ссылку и тело. Мне нужно извлечь ключевые моменты из каждой из этих новостей.

Первое, что бросается в глаза, это то, что Google на конференции I/O 2025 представила Gemma 3n. Это основная новость. В первой новости говорится, что модель работает с 2 ГБ памяти и без подключения к облаку. Вторая новость упоминает, что модель предназначена для мобильных устройств. Третья новость говорит о локальном использовании и поддержке разных форматов, таких как текст, изображения, видео и аудио. Четвертая новость упоминает, что это мультимодальная модель. Пятая новость подчеркивает легкость и скорость модели для работы на устройствах. Шестая новость говорит о расширении семейства Gemma, включая Med

In [72]:
print(answer['choices'][0]['message']['content'])



Вот краткое резюме новостей о модели **Gemma 3n**:

### Основные новости:
1. **Представлена на Google I/O 2025**  
   - Google на конференции I/O 2025 анонсировала модель **Gemma 3n** — новую версию семейства Gemma, предназначенную для локального использования на мобильных устройствах, ноутбуках и планшетах.

2. **Мобильная оптимизация**  
   - Модель работает с **2 ГБ ОЗУ**, что делает её подходящей для устройств с ограниченными ресурсами.  
   - Поддержка **140 языков** и возможность работы **без подключения к облаку**.

3. **Мультимодальность**  
   - Поддержка **текста, изображений, видео и аудио**.  
   - Включает функции, такие как **RAG (Retrieval-Augmented Generation)** и **мультимодальные трансформеры**.

4. **Расширение семейства Gemma**  
   - Введены новые версии: **MedGemma** (для медицинских приложений) и **SignGemma** (для перевода жестов).  
   - Семейство Gemma теперь включает модели для **здравоохранения, доступности и локальных решений**.

5. **Высокая производител